## Fixed Data:

In [2]:
pip install fredapi

In [3]:
import json
import requests
import pandas as pd
from fredapi import Fred

API_KEY = '1vGb0FOsg2hEIGrEhpueLCcWd1Y'
FRED_API_KEY = 'b13a905b98b5dfc1f3910d6a0d875ae3'

rootUrl = 'https://api.glassnode.com'

symbolList = ['BTC', 'ETH', 'LTC', 'AAVE', 'ABT', 'AMPL', 'ANT', 'ARMOR', 'BADGER', 'BAL', 'BAND', 'BAT', 'BIX', 'BNT', 'BOND', 'BRD', 'BUSD', 'BZRX', 'CELR', 'CHSB', 'CND', 'COMP', 'CREAM', 'CRO', 'CRV', 'CVC', 'CVP', 'DAI', 'DDX', 'DENT', 'DGX', 'DHT', 'DMG', 'DODO', 'DOUGH', 'DRGN', 'ELF', 'ENG', 'ENJ', 'EURS', 'FET', 'FTT', 'FUN', 'GNO', 'GUSD', 'HEGIC', 'HOT', 'HPT', 'HT', 'HUSD', 'INDEX', 'KCS', 'LAMB', 'LBA', 'LDO', 'LEO', 'LINK', 'LOOM', 'LRC', 'MANA', 'MATIC',
              'MCB', 'MCO', 'MFT', 'MIR', 'MKR', 'MLN', 'MTA', 'MTL', 'MX', 'NDX', 'NEXO', 'NFTX', 'NMR', 'Nsure', 'OCEAN', 'OKB', 'OMG', 'PAY', 'PERP', 'PICKLE', 'PNK', 'PNT', 'POLY', 'POWR', 'PPT', 'QASH', 'QKC', 'QNT', 'RDN', 'REN', 'REP', 'RLC', 'ROOK', 'RPL', 'RSR', 'SAI', 'SAN', 'SNT', 'SNX', 'STAKE', 'STORJ', 'sUSD', 'SUSHI', 'TEL', 'TOP', 'UBT', 'UMA', 'UNI', 'USDC', 'USDK', 'USDP', 'USDT', 'UTK', 'VERI', 'WaBi', 'WAX', 'WBTC', 'WETH', 'wNXM', 'WTC', 'YAM', 'YFI', 'ZRX']

symbolListShort = ['BTC', 'ETH', 'LTC']

endPoints = {
    'Price': '/v1/metrics/market/price_usd_close',
    'MarketCap': '/v1/metrics/market/marketcap_usd',
    'TransactionCount': '/v1/metrics/transactions/count',
    'ActiveAddress': '/v1/metrics/addresses/active_count',
}

## <font color='red'> It seems the risk-free rate is daily rate. is it in percentage or in unit 1? need to adjust to weekly rate and select Monday

In [13]:
fred = Fred(api_key=FRED_API_KEY)
Risk_free_rate = fred.get_series('DGS1MO')
Risk_free_rate = Risk_free_rate.to_frame()
Risk_free_rate = Risk_free_rate.reset_index()
Risk_free_rate = Risk_free_rate.rename(columns={'index':'Date', 0:'RiskFree'})

## Get Multiple Attributes for Multiple Coins:

In [12]:
### Define a function to pull data according to input values(coin, feature)
### After running this method, it should return pulled data, and also store it in memory named as "coin_feature"

def PullData(coin, feature):
    print(f"Pulling {coin}'s {feature} data")

    # Assemble full request url and get responses:
    reqUrl = rootUrl + endPoints[feature]
    res = requests.get(reqUrl,
                       params={'a': coin, 'api_key': API_KEY, 'i':'1w'})
    data_raw = pd.read_json(res.text, convert_dates=['t'])

    # Rename column "v" according to features
    data = data_raw.rename(columns={"v" : f"{feature}", 't':'Date'})

    # Turn renamed dataframe into a global variable for future using
    name = f"{coin}_{feature}"
    globals()[name] = data

    # Return a dataframe
    return data

In [65]:
### Get all features(listed in endPoints) of all coins(listed in coinList) 
def PullStart(coinList):

    from functools import reduce

    for i in coinList:
        print(f"Start Pulling {i}'s data:")

        # Preparation for merging
        frameName = f"{i}_Frames"
        locals()[frameName] = []

        # Get data
        for j in list(endPoints.keys()):
            locals()[frameName].append(PullData(i, j))

        # Merge to single dataframe
        print('Merging Dataframes...')
        dfName = f"{i}_All"
        # Outer merge to ensure not missing any values.
        locals()[dfName] = reduce(lambda  left,right: pd.merge(left,right, on='Date', how='outer'), locals()[frameName]) 
        
        # Left merege Fred to ensure all the records in api stored.
        locals()[dfName] = locals()[dfName].merge(Risk_free_rate,how = 'left').sort_values(by=['Date'])

        ## Imputation for NA values -- to be resolved
        #
        #
        ##Imputation for NA values -- to be resolved

        # Add Return Column(52 records for testing)
        temp = locals()[dfName][-53:-1].reset_index()
    
        locals()[dfName] = locals()[dfName][-52:].reset_index().drop('index', axis=1)

        locals()[dfName]['Return'] = (locals()[dfName]['Price'] - temp['Price'])/temp['Price']

        locals()[dfName]['Asset'] = i

        globals()[dfName] = locals()[dfName]

        print(f"{i} data successfully pulled\n")
    print('\nALL DATA SUCCESSFULLY PULLED!')

## <font color='red'> Imputation for NA values -- to be resolved

## MOM Factor

### Get LargeCap Sample(maket cap > 10000)

In [93]:
# Check if a coin is Large and get LargeCoin list
largeList = []
for i in symbolList:
    if PullData(i, 'MarketCap').iloc[-1,1] > 1000000:
        largeList.append(i)

Pulling BTC's MarketCap data
Pulling ETH's MarketCap data
Pulling LTC's MarketCap data
Pulling AAVE's MarketCap data
Pulling ABT's MarketCap data
Pulling AMPL's MarketCap data
Pulling ANT's MarketCap data
Pulling ARMOR's MarketCap data
Pulling BADGER's MarketCap data
Pulling BAL's MarketCap data
Pulling BAND's MarketCap data
Pulling BAT's MarketCap data
Pulling BIX's MarketCap data
Pulling BNT's MarketCap data
Pulling BOND's MarketCap data
Pulling BRD's MarketCap data
Pulling BUSD's MarketCap data
Pulling BZRX's MarketCap data
Pulling CELR's MarketCap data
Pulling CHSB's MarketCap data
Pulling CND's MarketCap data
Pulling COMP's MarketCap data
Pulling CREAM's MarketCap data
Pulling CRO's MarketCap data
Pulling CRV's MarketCap data
Pulling CVC's MarketCap data
Pulling CVP's MarketCap data
Pulling DAI's MarketCap data
Pulling DDX's MarketCap data
Pulling DENT's MarketCap data
Pulling DGX's MarketCap data
Pulling DHT's MarketCap data
Pulling DMG's MarketCap data
Pulling DODO's MarketCap d

In [94]:
PullStart(largeList)

Start Pulling BTC's data:
Pulling BTC's Price data
Pulling BTC's MarketCap data
Pulling BTC's TransactionCount data
Pulling BTC's ActiveAddress data
Merging Dataframes...
BTC data successfully pulled

Start Pulling ETH's data:
Pulling ETH's Price data
Pulling ETH's MarketCap data
Pulling ETH's TransactionCount data
Pulling ETH's ActiveAddress data
Merging Dataframes...
ETH data successfully pulled

Start Pulling LTC's data:
Pulling LTC's Price data
Pulling LTC's MarketCap data
Pulling LTC's TransactionCount data
Pulling LTC's ActiveAddress data
Merging Dataframes...
LTC data successfully pulled

Start Pulling AAVE's data:
Pulling AAVE's Price data
Pulling AAVE's MarketCap data
Pulling AAVE's TransactionCount data
Pulling AAVE's ActiveAddress data
Merging Dataframes...
AAVE data successfully pulled

Start Pulling ABT's data:
Pulling ABT's Price data
Pulling ABT's MarketCap data
Pulling ABT's TransactionCount data
Pulling ABT's ActiveAddress data
Merging Dataframes...
ABT data successful

In [95]:
def getLargeCap():

    largeCapSample = pd.DataFrame(columns = LTC_All.columns.values)  # can be improved

    largeCapSample['Asset'] = ''
    from functools import reduce

    for i in largeList:
        dfName = f"{i}_All" 
        largeCapSample = largeCapSample.append(globals()[dfName])
    
    return largeCapSample

In [96]:
largeCapSample = getLargeCap()

In [97]:
largeCapSample

,Date,Price,MarketCap,TransactionCount,ActiveAddress,RiskFree,Return,Asset
0,2021-04-26,56613.202648,1.058532e+12,1861992.0,5778871,0.02,0.154432,BTC
1,2021-05-03,58218.566077,1.088991e+12,2038316.0,6622607,0.02,0.028357,BTC
2,2021-05-10,46406.331544,8.683518e+11,1908851.0,6062366,0.02,-0.202895,BTC
3,2021-05-17,34888.678313,6.530094e+11,1607054.0,4960283,0.00,-0.248191,BTC
4,2021-05-24,35639.981455,6.672647e+11,1508758.0,4981835,0.01,0.021534,BTC
...,...,...,...,...,...,...,...,...
47,2022-03-21,0.725993,6.152760e+08,0.0,1603,0.22,0.319245,ZRX
48,2022-03-28,0.910632,7.717570e+08,0.0,835,0.19,0.254326,ZRX
49,2022-04-04,0.754619,6.395366e+08,0.0,1284,0.18,-0.171324,ZRX
50,2022-04-11,0.686854,5.821057e+08,0.0,1266,0.22,-0.089801,ZRX


### Reshape Dataframe

In [106]:
returnAll = largeCapSample.loc[:,['Date', 'Return', 'Asset']].pivot(index="Date", columns=["Asset"], values='Return').reset_index().rename_axis(None, axis=1).sort_values('Date')
returnAll.head()

,Date,AAVE,ABT,AMPL,ANT,ARMOR,BADGER,BAL,BAND,BAT,...,VERI,WAX,WBTC,WTC,WaBi,YAM,YFI,ZRX,sUSD,wNXM
0,2021-04-26,0.393008,0.148764,-0.073592,0.219156,0.028645,0.450839,0.282477,0.307409,0.229047,...,-0.047122,NaN,0.149613,0.326124,0.541436,0.153995,0.214523,0.394776,-0.003986,0.191180
1,2021-05-03,-0.049263,0.022970,0.112456,0.069679,0.080898,-0.024938,0.079109,0.001570,0.126182,...,0.406858,NaN,0.032826,0.141704,0.070760,-0.145964,0.102180,0.046813,0.010270,0.348608
2,2021-05-10,0.143665,-0.171471,-0.281857,-0.244447,-0.323333,-0.256346,-0.182583,-0.204940,-0.194722,...,0.062182,NaN,-0.204564,-0.164128,-0.319218,0.064864,0.234226,-0.195655,-0.007335,-0.150428
3,2021-05-17,-0.435925,-0.482439,-0.357192,-0.560776,-0.456955,-0.597872,-0.523110,-0.514725,-0.504094,...,-0.447383,NaN,-0.245743,-0.593304,-0.551696,-0.382593,-0.499353,-0.486727,-0.019204,-0.410748
4,2021-05-24,0.094413,0.379361,0.464830,0.228098,0.103387,0.117416,0.069808,0.085198,0.244939,...,0.188722,NaN,0.019047,0.204790,0.212288,-0.279703,0.272157,0.161636,0.005509,0.241871


### Calculate Momentum Matrix(return 1-week.)

In [107]:
momMatrix = returnAll.copy()
momMatrix['Date'] = momMatrix['Date'].shift(-1)
momMatrix = momMatrix.drop(momMatrix.tail(1).index)
momMatrix.head()

,Date,AAVE,ABT,AMPL,ANT,ARMOR,BADGER,BAL,BAND,BAT,...,VERI,WAX,WBTC,WTC,WaBi,YAM,YFI,ZRX,sUSD,wNXM
0,2021-05-03,0.393008,0.148764,-0.073592,0.219156,0.028645,0.450839,0.282477,0.307409,0.229047,...,-0.047122,NaN,0.149613,0.326124,0.541436,0.153995,0.214523,0.394776,-0.003986,0.191180
1,2021-05-10,-0.049263,0.022970,0.112456,0.069679,0.080898,-0.024938,0.079109,0.001570,0.126182,...,0.406858,NaN,0.032826,0.141704,0.070760,-0.145964,0.102180,0.046813,0.010270,0.348608
2,2021-05-17,0.143665,-0.171471,-0.281857,-0.244447,-0.323333,-0.256346,-0.182583,-0.204940,-0.194722,...,0.062182,NaN,-0.204564,-0.164128,-0.319218,0.064864,0.234226,-0.195655,-0.007335,-0.150428
3,2021-05-24,-0.435925,-0.482439,-0.357192,-0.560776,-0.456955,-0.597872,-0.523110,-0.514725,-0.504094,...,-0.447383,NaN,-0.245743,-0.593304,-0.551696,-0.382593,-0.499353,-0.486727,-0.019204,-0.410748
4,2021-05-31,0.094413,0.379361,0.464830,0.228098,0.103387,0.117416,0.069808,0.085198,0.244939,...,0.188722,NaN,0.019047,0.204790,0.212288,-0.279703,0.272157,0.161636,0.005509,0.241871


### Calculate MOM Percentile(30%, 70%)

In [108]:
# Only elect Assets(BTC, ETH...) columns
col = []
for colName in momMatrix.columns:
     if colName not in ['Date','MOM30p','MOM70p']:
         col.append(colName)
momMatrix[col].head()

,AAVE,ABT,AMPL,ANT,ARMOR,BADGER,BAL,BAND,BAT,BIX,...,VERI,WAX,WBTC,WTC,WaBi,YAM,YFI,ZRX,sUSD,wNXM
0,0.393008,0.148764,-0.073592,0.219156,0.028645,0.450839,0.282477,0.307409,0.229047,0.596650,...,-0.047122,NaN,0.149613,0.326124,0.541436,0.153995,0.214523,0.394776,-0.003986,0.191180
1,-0.049263,0.022970,0.112456,0.069679,0.080898,-0.024938,0.079109,0.001570,0.126182,-0.168886,...,0.406858,NaN,0.032826,0.141704,0.070760,-0.145964,0.102180,0.046813,0.010270,0.348608
2,0.143665,-0.171471,-0.281857,-0.244447,-0.323333,-0.256346,-0.182583,-0.204940,-0.194722,-0.131835,...,0.062182,NaN,-0.204564,-0.164128,-0.319218,0.064864,0.234226,-0.195655,-0.007335,-0.150428
3,-0.435925,-0.482439,-0.357192,-0.560776,-0.456955,-0.597872,-0.523110,-0.514725,-0.504094,-0.509419,...,-0.447383,NaN,-0.245743,-0.593304,-0.551696,-0.382593,-0.499353,-0.486727,-0.019204,-0.410748
4,0.094413,0.379361,0.464830,0.228098,0.103387,0.117416,0.069808,0.085198,0.244939,0.010949,...,0.188722,NaN,0.019047,0.204790,0.212288,-0.279703,0.272157,0.161636,0.005509,0.241871


In [109]:
# 30% pencentile
momMatrix[col].quantile(.3, axis=1).head()

0    0.154039
1   -0.020051
2   -0.211393
3   -0.517193
4    0.070218
Name: 0.3, dtype: float64

In [110]:
momMatrix['MOM30p']=momMatrix[col].quantile(.3, axis=1)
momMatrix['MOM70p']=momMatrix[col].quantile(.7, axis=1)
momMatrix.head()

,Date,AAVE,ABT,AMPL,ANT,ARMOR,BADGER,BAL,BAND,BAT,...,WBTC,WTC,WaBi,YAM,YFI,ZRX,sUSD,wNXM,MOM30p,MOM70p
0,2021-05-03,0.393008,0.148764,-0.073592,0.219156,0.028645,0.450839,0.282477,0.307409,0.229047,...,0.149613,0.326124,0.541436,0.153995,0.214523,0.394776,-0.003986,0.191180,0.154039,0.321489
1,2021-05-10,-0.049263,0.022970,0.112456,0.069679,0.080898,-0.024938,0.079109,0.001570,0.126182,...,0.032826,0.141704,0.070760,-0.145964,0.102180,0.046813,0.010270,0.348608,-0.020051,0.100052
2,2021-05-17,0.143665,-0.171471,-0.281857,-0.244447,-0.323333,-0.256346,-0.182583,-0.204940,-0.194722,...,-0.204564,-0.164128,-0.319218,0.064864,0.234226,-0.195655,-0.007335,-0.150428,-0.211393,-0.070300
3,2021-05-24,-0.435925,-0.482439,-0.357192,-0.560776,-0.456955,-0.597872,-0.523110,-0.514725,-0.504094,...,-0.245743,-0.593304,-0.551696,-0.382593,-0.499353,-0.486727,-0.019204,-0.410748,-0.517193,-0.370030
4,2021-05-31,0.094413,0.379361,0.464830,0.228098,0.103387,0.117416,0.069808,0.085198,0.244939,...,0.019047,0.204790,0.212288,-0.279703,0.272157,0.161636,0.005509,0.241871,0.070218,0.224148


In [111]:
# reshape for later use
momPt = momMatrix[['MOM30p','MOM70p']]
momMatrix = momMatrix.drop(['MOM30p','MOM70p'], axis=1)

### Calculate Momentum portfolio returns

In [112]:
# delete the first 1 row of returnAll to match the momMatrix date
returnAll = returnAll.drop(returnAll.head(1).index).reset_index().drop('index', axis=1)
returnAll.head()

,Date,AAVE,ABT,AMPL,ANT,ARMOR,BADGER,BAL,BAND,BAT,...,VERI,WAX,WBTC,WTC,WaBi,YAM,YFI,ZRX,sUSD,wNXM
0,2021-05-03,-0.049263,0.022970,0.112456,0.069679,0.080898,-0.024938,0.079109,0.001570,0.126182,...,0.406858,NaN,0.032826,0.141704,0.070760,-0.145964,0.102180,0.046813,0.010270,0.348608
1,2021-05-10,0.143665,-0.171471,-0.281857,-0.244447,-0.323333,-0.256346,-0.182583,-0.204940,-0.194722,...,0.062182,NaN,-0.204564,-0.164128,-0.319218,0.064864,0.234226,-0.195655,-0.007335,-0.150428
2,2021-05-17,-0.435925,-0.482439,-0.357192,-0.560776,-0.456955,-0.597872,-0.523110,-0.514725,-0.504094,...,-0.447383,NaN,-0.245743,-0.593304,-0.551696,-0.382593,-0.499353,-0.486727,-0.019204,-0.410748
3,2021-05-24,0.094413,0.379361,0.464830,0.228098,0.103387,0.117416,0.069808,0.085198,0.244939,...,0.188722,NaN,0.019047,0.204790,0.212288,-0.279703,0.272157,0.161636,0.005509,0.241871
4,2021-05-31,0.126079,-0.039411,0.076882,0.345243,-0.040499,0.302944,0.054685,0.090265,0.082609,...,0.046409,NaN,0.005559,-0.025069,0.090076,0.100535,0.028074,0.128756,0.000937,0.052810


In [113]:
# initialize df for low, medium and high MOM
lowMom = pd.DataFrame(columns = ['Date', 'LowMomentum'])
lowMom['Date'] = momMatrix['Date']

medMom = pd.DataFrame(columns = ['Date', 'MediumMomentum'])
medMom['Date'] = momMatrix['Date']

highMom = pd.DataFrame(columns = ['Date', 'HighMomentum'])
highMom['Date'] = momMatrix['Date']

In [114]:
# iterate through whole dataframe to get low, medium and high MOM
lowList = []
medList = []
highList = []
for i in range(0, momMatrix.shape[0]):
    for j in range(1, momMatrix.shape[1]):
        if momMatrix.iloc[i,j] < momPt.iloc[i,0]:
            lowList.append(returnAll.iloc[i,j])
        elif (momPt.iloc[i,0] < momMatrix.iloc[i,j]) and (momMatrix.iloc[i,j] < momPt.iloc[i,1]):
            medList.append(returnAll.iloc[i,j])
        elif momMatrix.iloc[i,j] > momPt.iloc[i,1]:
            highList.append(returnAll.iloc[i,j])
    lowMom.iloc[i,1] = sum(lowList) / len(lowList)
    medMom.iloc[i,1] = sum(medList) / len(medList)
    highMom.iloc[i,1] = sum(highList) / len(highList)

In [116]:
from functools import reduce
momLMH = reduce(lambda  left,right: pd.merge(left,right, on='Date', how='outer'), [lowMom, medMom, highMom])
momLMH.head()

,Date,LowMomentum,MediumMomentum,HighMomentum
0,2021-05-03,0.028317,0.084243,0.167024
1,2021-05-10,-0.051756,-0.020802,0.021574
2,2021-05-17,-0.19436,-0.1585,-0.089848
3,2021-05-24,-0.084516,-0.072168,-0.054362
4,2021-05-31,-0.055801,-0.036919,-0.023456


### Calculate Momentum Factor

In [117]:
momFactor = pd.DataFrame(columns = ['Date', 'MOM_LTW'])
momFactor['Date'] = momMatrix['Date']
momFactor['MOM_LTW'] = momLMH['HighMomentum'] - momLMH['LowMomentum']

In [118]:
momFactor.head(20)

,Date,MOM_LTW
0,2021-05-03,0.138707
1,2021-05-10,0.07333
2,2021-05-17,0.104512
3,2021-05-24,0.030154
4,2021-05-31,0.032345
5,2021-06-07,0.018745
6,2021-06-14,0.018277
7,2021-06-21,0.019533
8,2021-06-28,-0.000888
9,2021-07-05,-0.001338
